In [2]:
%load_ext autoreload
%autoreload 2
import data
import model as m
import util
import os
import re
import cv2
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

for model_name in ["resnet101"]:
    #hyperparam
    batch_size = 16
    num_epochs = 128
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    learning_rate = 0.0001
    reg_lambda = 0.0001

    def dir_to_img_tensor(*paths):
        #return batch_size x channel x width x height
        layers_in_station = {}
        for path in paths:
            if not path.endswith('/'):
                path += '/'
            files = os.listdir(path)
            for file in files:
                file = path+file
                expression = r"station_(?P<label>\w+)_(?P<station_id>\d+)_(?P<zoom>\d+)_(?P<mode>\w+).png"
                regex = re.compile(expression)
                label,station_id,mode,zoom = regex.findall(file)[0]
                
                if label=="true":
                    label = 1.
                elif label=="false":
                    label = 0.
                
                img = cv2.imread(file)
                #img = np.transpose(img, (2,0,1))

                if not (station_id, zoom) in layers_in_station.keys():
                    station_data = {'X':[img],'y':label}
                    layers_in_station[(station_id, zoom)] = station_data
                else:
                    layers_in_station[(station_id, zoom)]['X'].append(img) #image file은 특정 순서대로 저장됨

        X = np.array([np.concatenate(station_info['X'],axis=2) for station_info in layers_in_station.values()])
        y = np.array([station_info['y'] for station_info in layers_in_station.values()]).reshape([-1,1])

        return X, y
    #load data from directory
    X, y = dir_to_img_tensor("datasets")
    print("complete data composition")
    #make torch dataset
    transform = data.compose_transform(X.shape[-1],img_size=(X.shape[1],X.shape[2]))
    dataset = data.MapDataset(X,y,transform)
    img_shape = dataset[0][0].shape

    train_dataset, valid_dataset, test_dataset = util.split_data(dataset, 0.8,0.1,0.1)

    #data loader
    train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size, shuffle=True)

    #model
    model = m.load_torch_hub(model_name)
    print(f"{model_name} : {len(list(model.children()))}")
    model = m.fit_model_structure(model, img_shape[0], 1)

    loss_function = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=reg_lambda)
    print("Start training")
    acc = m.train(model, train_loader, valid_loader, loss_function, optimizer, num_epochs, device, 1, model_name)
    print("Start testing")
    pred, correct = m.predict(model, test_loader, device)

    fd = open(f"result/result_{model_name}.txt","a")
    fd.write(str(util.evaluation_metric(pred, correct)))
    print("\n")
    fd.write(str(pred))
    print("\n")
    fd.write(str(correct))
    print("\n")
    fd.close()
    print("complete writing")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
complete data composition


Using cache found in C:\Users\netis/.cache\torch\hub\pytorch_vision_v0.10.0


resnet101 : 10
Start training


RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 8.00 GiB total capacity; 7.14 GiB already allocated; 0 bytes free; 7.30 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF